# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.3137, -0.3088, -0.0265,  0.0914,  0.0738,  0.1713, -0.0730,  0.0644,
         -0.1429,  0.1958],
        [ 0.2520, -0.4221, -0.0076,  0.1561,  0.0557,  0.0843, -0.0273,  0.0482,
         -0.1979,  0.2274]], grad_fn=<AddmmBackward0>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[ 0.0484,  0.1010, -0.0442,  0.0688, -0.2698, -0.1924,  0.0629,  0.0604,
         -0.2581, -0.1574],
        [ 0.0757,  0.1118,  0.0371,  0.1509, -0.2893, -0.1181,  0.0592, -0.0285,
         -0.2854,  0.0210]], grad_fn=<AddmmBackward0>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0498, -0.1249,  0.2453, -0.2897,  0.0960, -0.0883, -0.4407,  0.1057,
         -0.2348,  0.0971],
        [ 0.0589, -0.1916,  0.2488, -0.3291, -0.0067,  0.0711, -0.4173, -0.1374,
         -0.2823, -0.0459]], grad_fn=<AddmmBackward0>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.2734, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1406, grad_fn=<SumBackward0>)